In [ ]:
import bpy
import os

#change these per class
image_name = "local_cocoa"
output_path = "C:\Datasets\local cocoa\output"
input_src_path = "C:\Datasets\local cocoa\source"
input_dm_path = "C:\Datasets\local cocoa\dm"

#whatever the material name is
material_name = "busy_flowers"
material = bpy.data.materials[material_name]
nodes = material.node_tree.nodes
links = material.node_tree.links

#custom names of the nodes
image_node_name = "base_img"
displacement_node_name = "displacement_img"

#empty node references to fill
displacement_node,image_node = None,None

#Look for the nodes in our list of nodes
for node in nodes:
    if node.label == image_node_name:
        print("image node found!")
        image_node = node
        
    elif node.label == displacement_node_name:
        print("displacement node found!")
        displacement_node = node

#set up automated rendering
def render(dis,img,input_src,input_dm,output):
    if dis and img:
        print("Both nodes found. Beginning renders")
        displacement_images = [img for img in os.listdir(input_dm)] 
        src_images = [img for img in os.listdir(input_src)]
        
        if len(displacement_images) != len(src_images):
            print("[Error]: Number of images mismatch!")
        else:
            for d_im_name,s_im_name in zip(displacement_images,src_images):
                #load displacement image
                disp_img_path = os.path.join(input_dm,d_im_name)
                disp_img = bpy.data.images.load(disp_img_path)
                
                #load source 
                source_img_path = os.path.join(input_src,s_im_name)
                source_img = bpy.data.images.load(source_img_path)
                
                #set the images in the nodes
                dis.image = disp_img
                img.image = source_img
                
                #set render output path and file name
                bpy.context.scene.render.filepath = os.path.join(output_path,f"{image_name}_{d_im_name}_{s_im_name}")
                
                #render image
                bpy.ops.render.render(write_still=True)
                
            print(f"[INFO]: {len(src_images)} Renders Complete")
    else:
        print("Missing node information")
        
    
render(displacement_node,image_node,input_src_path,input_dm_path,output_path)
